In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
train_dataset = torchvision.datasets.MNIST(root="../../data" , train=True, transform= transforms.ToTensor(), download=True )
test_dataset = torchvision.datasets.MNIST(root="../../data", train= False, transform= transforms.ToTensor(), download=True)

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False)

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
model = NeuralNetwork(input_size, hidden_size, num_classes).to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2775
Epoch [1/5], Step [200/600], Loss: 0.2926
Epoch [1/5], Step [300/600], Loss: 0.1252
Epoch [1/5], Step [400/600], Loss: 0.1005
Epoch [1/5], Step [500/600], Loss: 0.1589
Epoch [1/5], Step [600/600], Loss: 0.1313
Epoch [2/5], Step [100/600], Loss: 0.1201
Epoch [2/5], Step [200/600], Loss: 0.2019
Epoch [2/5], Step [300/600], Loss: 0.1584
Epoch [2/5], Step [400/600], Loss: 0.2679
Epoch [2/5], Step [500/600], Loss: 0.0654
Epoch [2/5], Step [600/600], Loss: 0.0410
Epoch [3/5], Step [100/600], Loss: 0.0282
Epoch [3/5], Step [200/600], Loss: 0.0378
Epoch [3/5], Step [300/600], Loss: 0.0462
Epoch [3/5], Step [400/600], Loss: 0.0312
Epoch [3/5], Step [500/600], Loss: 0.0671
Epoch [3/5], Step [600/600], Loss: 0.0762
Epoch [4/5], Step [100/600], Loss: 0.0332
Epoch [4/5], Step [200/600], Loss: 0.0267
Epoch [4/5], Step [300/600], Loss: 0.0482
Epoch [4/5], Step [400/600], Loss: 0.0496
Epoch [4/5], Step [500/600], Loss: 0.0317
Epoch [4/5], Step [600/600], Loss:

In [12]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 97.87 %


In [13]:
torch.save(model.state_dict(), 'FeedForwardModel.ckpt')